In [1]:
import pandas as pd
pd.options.display.max_columns = None

import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import tensorflow as tf

CREATE_SET = False

In [2]:
tf.keras.backend.clear_session()
from tensorflow import keras
from tensorflow.keras import layers

### Retrieve the data 

In [3]:
#Load all Files (they must be in data directory in a brother directory of the notebook)
data_load = {
    'dipole_moments': pd.read_csv('./data/dipole_moments.csv'),
    'magnetic_shielding_tensors': pd.read_csv('./data/magnetic_shielding_tensors.csv'),
    'mulliken_charges': pd.read_csv('./data/mulliken_charges.csv'),
    'potential_energy': pd.read_csv('./data/potential_energy.csv'),
    'sample_submission': pd.read_csv('./data/sample_submission.csv'),
    'scalar_coupling_contributions': pd.read_csv('./data/scalar_coupling_contributions.csv'),
    'structures': pd.read_csv('./data/structures.csv'),
    'train': pd.read_csv('./data/train.csv'), 
    'test': pd.read_csv('./data/test.csv')
    }

In [4]:
train_all = data_load['train']
MAX_MOL_ATOMS_NB = max(train_all.atom_index_0.max(),train_all.atom_index_1.max()) + 1
COUPLING_TYPE_NB = 8

### Optional

In [5]:
#Confirmation with the molecule structure
structures = data_load['structures']

structures_col = structures.columns


# Create one_hot_encoding for the atom type
structures = pd.get_dummies(structures, prefix = 'atom', columns = ['atom'])
structures.head()

,molecule_name,atom_index,x,y,z,atom_C,atom_F,atom_H,atom_N,atom_O
0,dsgdb9nsd_000001,0,-0.012698,1.085804,0.008001,1,0,0,0,0
1,dsgdb9nsd_000001,1,0.002150,-0.006031,0.001976,0,0,1,0,0
2,dsgdb9nsd_000001,2,1.011731,1.463751,0.000277,0,0,1,0,0
3,dsgdb9nsd_000001,3,-0.540815,1.447527,-0.876644,0,0,1,0,0
4,dsgdb9nsd_000001,4,-0.523814,1.437933,0.906397,0,0,1,0,0


In [6]:
# No shuffle to the samples, this will be done later
train_all = data_load['train']
train_all =  pd.get_dummies(train_all, prefix = 'coupling_type', columns = ['type'])
train_col = train_all.columns

# Add the dipole_moments, per molecule
dipole_moments = data_load['dipole_moments'].rename(columns = {"X": 'DM_X', "Y": "DM_Y", "Z": "DM_Z" })
train_all = train_all.merge(dipole_moments, on = ['molecule_name'])

# Add the potential energy, per molecule
potential_energy = data_load['potential_energy']
train_all = train_all.merge(potential_energy, on = ['molecule_name'])


# Add the Mulliken charges, per atom
mulliken_charges = data_load['mulliken_charges']
train_all = train_all.merge(mulliken_charges, 
                            left_on = ['molecule_name', 'atom_index_0'], 
                            right_on = ['molecule_name', 'atom_index'] )
train_all = train_all.merge(mulliken_charges, 
                            left_on = ['molecule_name', 'atom_index_1'], 
                            right_on = ['molecule_name', 'atom_index'], suffixes = ('_atom_0', '_atom_1') 
                           ).drop(['atom_index_atom_0', 'atom_index_atom_1'], axis = 1)

# Add magnetic shileding, per atom
magnetic_shielding_tensors = data_load['magnetic_shielding_tensors']
train_all = train_all.merge(magnetic_shielding_tensors, 
                            left_on = ['molecule_name', 'atom_index_0'], 
                            right_on = ['molecule_name', 'atom_index'] )
train_all = train_all.merge(magnetic_shielding_tensors, 
                            left_on = ['molecule_name', 'atom_index_1'], 
                            right_on = ['molecule_name', 'atom_index'], suffixes = ('_atom_0', '_atom_1') 
                           ).drop(['atom_index_atom_0', 'atom_index_atom_1'], axis = 1)


train_all.head(5)

,id,molecule_name,atom_index_0,atom_index_1,scalar_coupling_constant,coupling_type_1JHC,coupling_type_1JHN,coupling_type_2JHC,coupling_type_2JHH,coupling_type_2JHN,coupling_type_3JHC,coupling_type_3JHH,coupling_type_3JHN,DM_X,DM_Y,DM_Z,potential_energy,mulliken_charge_atom_0,mulliken_charge_atom_1,XX_atom_0,YX_atom_0,ZX_atom_0,XY_atom_0,YY_atom_0,ZY_atom_0,XZ_atom_0,YZ_atom_0,ZZ_atom_0,XX_atom_1,YX_atom_1,ZX_atom_1,XY_atom_1,YY_atom_1,ZY_atom_1,XZ_atom_1,YZ_atom_1,ZZ_atom_1
0,0,dsgdb9nsd_000001,1,0,84.8076,1,0,0,0,0,0,0,0,0.0,0.0,0.0,-40.52368,0.133921,-0.535689,31.3410,-1.2317,4.0544,-1.2317,28.9546,-1.7173,4.0546,-1.7173,34.0861,195.3150,0.0000,-0.0001,0.0000,195.3170,0.0007,-0.0001,0.0007,195.3170
1,4,dsgdb9nsd_000001,2,0,84.8074,1,0,0,0,0,0,0,0,0.0,0.0,0.0,-40.52368,0.133922,-0.535689,31.5814,1.2173,-4.1474,1.2173,28.9036,-1.6036,-4.1476,-1.6036,33.8967,195.3150,0.0000,-0.0001,0.0000,195.3170,0.0007,-0.0001,0.0007,195.3170
2,7,dsgdb9nsd_000001,3,0,84.8093,1,0,0,0,0,0,0,0,0.0,0.0,0.0,-40.52368,0.133923,-0.535689,31.5172,4.1086,1.2723,4.1088,33.9068,1.6950,1.2724,1.6951,28.9579,195.3150,0.0000,-0.0001,0.0000,195.3170,0.0007,-0.0001,0.0007,195.3170
3,9,dsgdb9nsd_000001,4,0,84.8095,1,0,0,0,0,0,0,0,0.0,0.0,0.0,-40.52368,0.133923,-0.535689,31.4029,-4.0942,-1.1793,-4.0944,34.0776,1.6259,-1.1795,1.6260,28.9013,195.3150,0.0000,-0.0001,0.0000,195.3170,0.0007,-0.0001,0.0007,195.3170
4,1,dsgdb9nsd_000001,1,2,-11.2570,0,0,0,1,0,0,0,0,0.0,0.0,0.0,-40.52368,0.133921,0.133922,31.3410,-1.2317,4.0544,-1.2317,28.9546,-1.7173,4.0546,-1.7173,34.0861,31.5814,1.2173,-4.1474,1.2173,28.9036,-1.6036,-4.1476,-1.6036,33.8967


Remark: besides train features, data are not provided in the test dataset.

Should we use those as generative features for the scalar coupling constant?"

## Data preparation

#### Function to prepare the global set

In [7]:
from timeit import default_timer as timer
import re


def create_coupling_per_mol(molecules, coupled_atoms):
    
    #0 Select all molecules in the batch first (to quicken the operations)
    molecules_name = coupled_atoms[['molecule_name']].values.reshape(-1)   
    molecules = molecules.loc[molecules['molecule_name'].isin(molecules_name)]

    #2 Prepare the features
    coupled_atoms.drop(['id'], axis = 1, inplace = True)

    coupling_id_col = ['molecule_name', 'atom_index_0', 'atom_index_1' ] 
    coupling_col = list(filter(re.compile("coupling_type.*").match,coupled_atoms.columns))
    outputs_atom_0_col = list(filter(re.compile(".*_atom_0").match,coupled_atoms.columns))
    outputs_atom_1_col = list(filter(re.compile(".*_atom_1").match,coupled_atoms.columns))
    outputs_mol_col = ['DM_X','DM_Y','DM_Z','potential_energy']
    
    #mean_col = ['x_mean', 'y_mean', 'z_mean']
    
    # Add type and coordinates for each atom_index_0 and atom_index_1
    df_tmp = coupled_atoms.merge(molecules, how='inner', 
                               left_on=['molecule_name', 'atom_index_0'], 
                               right_on=['molecule_name', 'atom_index'])
    df_features_dnn = df_tmp.merge(molecules, how='inner', 
                                          left_on=['molecule_name', 'atom_index_1'], 
                                          right_on=['molecule_name', 'atom_index'],
                                         suffixes=('_fa0', '_fa1')).drop(['atom_index_fa0','atom_index_fa1' ], axis = 1)
    
    # Return a tuple with 5 components grouped by molecule, in molecule order
    gb = df_features_dnn.groupby(['molecule_name'], sort = True, group_keys = False, as_index = True)
    
    coupling_pos_col = ['x_fa0' ,'y_fa0','z_fa0', 'x_fa1' ,'y_fa1','z_fa1']
    coupling_per_mol = gb.apply(lambda frame: frame[coupling_col + coupling_pos_col].to_numpy()).to_list()
    outputs_0 = gb.apply(lambda frame: frame[outputs_atom_0_col + outputs_mol_col].to_numpy()).to_list()
    outputs_1 = gb.apply(lambda frame: frame[outputs_atom_1_col + outputs_mol_col].to_numpy()).to_list()
    y_per_mol = gb.apply(lambda frame: frame['scalar_coupling_constant'].to_numpy()).to_list()
    
    # Return the molecule info per molecule, sorted by molecule order
    gb = molecules.groupby(['molecule_name'], sort = True, group_keys = False, as_index = True)
    molecules_output = gb.apply(lambda frame: frame.drop(['molecule_name', 'atom_index'], axis = 1).to_numpy()).to_list()
    
    return coupling_per_mol, y_per_mol, outputs_0, outputs_1, molecules_output

### Prepare the global train/eval sets (about 5 minutes)

In [8]:
from timeit import default_timer as timer
from sklearn.utils import shuffle 


if CREATE_SET:

    validation_ratio = 0.2

    # Retrieve the full list of molecules in the train/eval set
    molecule_list = train_all['molecule_name'].unique()

    # Split the train/eval sets on the molecule list
    train_mol, eval_mol = np.split(molecule_list, [int((1-validation_ratio)*len(molecule_list))])
    start = timer()

    # Select the samples belonging to the molecule train set (WARNING: there is no shuffle there!!)
    train_molecules = structures[structures['molecule_name'].isin(train_mol)]
    train_coupling, train_y, train_outputs_0, train_outputs_1, train_mols = create_coupling_per_mol(
                                        train_molecules,
                                        train_all.loc[train_all['molecule_name'].isin(train_mol)]
                                        )
    # Shuffle the samples by molecule
    shuffle(train_coupling, train_y, train_outputs_0, train_outputs_1, train_mols)

    train_stop = timer()

    # Select the samples belonging to the molecule eval set (WARNING: there is no shuffle there!!)
    eval_molecules = structures[structures['molecule_name'].isin(eval_mol)]
    eval_coupling, eval_y, eval_outputs_0, eval_outputs_1, eval_mols = create_coupling_per_mol(
                                        eval_molecules,
                                        train_all.loc[train_all['molecule_name'].isin(eval_mol)]
                                        )

    # Shuffle the samples by molecule
    shuffle(eval_coupling, eval_y, eval_outputs_0, eval_outputs_1, eval_mols)

    stop = timer()

    print('Number of samples: \nfor train set:', len(train_coupling), '\nfor eval set:', len(eval_coupling))
    print('Time for train / eval:', train_stop-start, '/', stop-train_stop)

#### To save the global set of molecules

In [9]:
# Step 1
import pickle

if CREATE_SET:
    global_data = {
        'train_coupling': train_coupling,
        'train_y': train_y,
        'train_outputs_0': train_outputs_0,
        'train_outputs_1': train_outputs_1,
        'train_mols': train_mols,
        'eval_coupling': eval_coupling,
        'eval_y': eval_y,
        'eval_outputs_0': eval_outputs_0,
        'eval_outputs_1': eval_outputs_1,
        'eval_mols': eval_mols,
        }

    # Step 2
    with open('train-eval-dump-2', 'wb') as dump_file:

        # Step 3
        pickle.dump(global_data, dump_file)
        
else:
    with open('train-eval-dump-2', 'rb') as dump_file:
        global_data = pickle.load(dump_file)

### Generation of the dataset

#### Creation of the train data set

In [10]:
# Alternative with masking and large batch (2500)

from timeit import default_timer as timer

MASKING_VALUE = 666

@tf.function
def treat_molecule_set(coupled_atoms_set, y, output_0, output_1, molecule):
    
    coupled_atoms_set = tf.convert_to_tensor(coupled_atoms_set)
    output_0 = tf.convert_to_tensor(output_0)
    output_1 = tf.convert_to_tensor(output_1)
    molecule = tf.convert_to_tensor(molecule)
    y = tf.convert_to_tensor(y)
    
    mol_atoms_nb = tf.shape(molecule)[0]
    init_molecule = molecule
    
    # Extend the molecule to 3D and transpose it 
    molecule = tf.reshape(molecule[:,:3], [-1, 3, 1])
    molecule = tf.transpose(molecule, [2, 1, 0])
    
    # We repeat the molecules and the mean values, so they have the same n_samples * n_atoms dimension
    molecule = tf.tile(molecule, [tf.shape(coupled_atoms_set)[0], 1, 1])

    pos_0_3d = tf.reshape(coupled_atoms_set[:, -6:-3], [-1, 3, 1])  
    pos_0_3d = tf.tile(pos_0_3d, [1, 1, mol_atoms_nb])
    
    pos_1_3d = tf.reshape(coupled_atoms_set[:, -3:], [-1, 3, 1])  
    pos_1_3d = tf.tile(pos_1_3d, [1, 1, mol_atoms_nb])
    
    
    # We calculate the distance 
    distance_0 = tf.reduce_sum(tf.square(tf.subtract(molecule, pos_0_3d)), 1)
    distance_1 = tf.reduce_sum(tf.square(tf.subtract(molecule, pos_1_3d)), 1)
    
    # This function defines for each sample the atoms of the molecule, sorted by decreasing order
    ordered_idx_0 = tf.argsort(
        distance_0,
        axis=1,
        direction='DESCENDING',
        stable=False,
        name=None
    )
    
    ordered_idx_1 = tf.argsort(
        distance_1,
        axis=1,
        direction='DESCENDING',
        stable=False,
        name=None
    )
    
    # We apply this sorting order to the initial molecule to calculate, for each couple of atoms, 
    # the atoms sorted by decreasing distance
    sort_mol_0 = tf.gather(
        init_molecule,
        ordered_idx_0
    )
    sort_mol_1 = tf.gather(
        init_molecule,
        ordered_idx_1
    )
    
    # Pad with 666 to keep the same size (is that even needed?)
    sort_mol_0 = tf.pad(sort_mol_0, [[0,0],[0,MAX_MOL_ATOMS_NB-mol_atoms_nb],[0,0]], constant_values=MASKING_VALUE)
    sort_mol_1 = tf.pad(sort_mol_1, [[0,0],[0,MAX_MOL_ATOMS_NB-mol_atoms_nb],[0,0]], constant_values=MASKING_VALUE)
    return { 'input_dnn': coupled_atoms_set, 
            'input_0': sort_mol_0,
            'input_1': sort_mol_1
        }, { 'y': y,
            'rnn_output_0': output_0,
            'rnn_output_1': output_1}

coupling = global_data['train_coupling']
y  = global_data['train_y']
outputs_0 = global_data['train_outputs_0']
outputs_1 = global_data['train_outputs_1']
mols = global_data['train_mols']


def gen_mol():
    for i in range(len(coupling)):     
        yield coupling[i], y[i], outputs_0[i], outputs_1[i], mols[i]

train_dataset = tf.data.Dataset.from_generator(
    gen_mol , (tf.float32, tf.float32, tf.float32, tf.float32, tf.float32), 
    (
        tf.TensorShape([None, 14]), 
        tf.TensorShape([None,]), 
        tf.TensorShape([None, 14]), 
        tf.TensorShape([None, 14]), 
        tf.TensorShape([None,8])
    ))

train_dataset = train_dataset.flat_map(
        lambda x, y, z0, z1, mol: tf.data.Dataset.from_tensor_slices(treat_molecule_set(x, y, z0, z1, mol))
    )
train_dataset = train_dataset.batch(2000,
    drop_remainder=True)
train_dataset = train_dataset.repeat()
train_dataset = train_dataset.prefetch(1)

train_dataset.take(1)

W0716 08:00:37.492140  8180 deprecation.py:323] From D:\Program_Files\Anaconda3\envs\tensorflow2\lib\site-packages\tensorflow\python\data\ops\dataset_ops.py:505: py_func (from tensorflow.python.ops.script_ops) is deprecated and will be removed in a future version.
Instructions for updating:
tf.py_func is deprecated in TF V2. Instead, there are two
    options available in V2.
    - tf.py_function takes a python function which manipulates tf eager
    tensors instead of numpy arrays. It's easy to convert a tf eager tensor to
    an ndarray (just call tensor.numpy()) but having access to eager tensors
    means `tf.py_function`s can use accelerators such as GPUs as well as
    being differentiable using a gradient tape.
    - tf.numpy_function maintains the semantics of the deprecated tf.py_func
    (it is not differentiable, and manipulates numpy arrays). It drops the
    stateful argument making all functions stateful.
    


<DatasetV1Adapter shapes: ({input_dnn: (2000, 14), input_0: (2000, None, 8), input_1: (2000, None, 8)}, {y: (2000,), rnn_output_0: (2000, 14), rnn_output_1: (2000, 14)}), types: ({input_dnn: tf.float32, input_0: tf.float32, input_1: tf.float32}, {y: tf.float32, rnn_output_0: tf.float32, rnn_output_1: tf.float32})>

In [11]:
# Alternative with batch of 1000

coupling_eval = global_data['eval_coupling']
y_eval  = global_data['eval_y']
outputs_0_eval = global_data['eval_outputs_0']
outputs_1_eval = global_data['eval_outputs_1']
mols_eval = global_data['eval_mols']

def gen_mol_eval():
    for i in range(len(coupling_eval)):     
        yield coupling_eval[i], y_eval[i], outputs_0_eval[i], outputs_1_eval[i], mols_eval[i]

eval_dataset = tf.data.Dataset.from_generator(
    gen_mol_eval , (tf.float32, tf.float32, tf.float32, tf.float32, tf.float32), 
    (
        tf.TensorShape([None, 14]), 
        tf.TensorShape([None,]), 
        tf.TensorShape([None, 14]), 
        tf.TensorShape([None, 14]), 
        tf.TensorShape([None,8])
    ))

eval_dataset = eval_dataset.flat_map(lambda x, y, z0, z1, mol: tf.data.Dataset.from_tensor_slices(treat_molecule_set(x, y, z0, z1, mol)))
eval_dataset = eval_dataset.batch(2000,
    drop_remainder=True)

eval_dataset = eval_dataset.repeat()
eval_dataset = eval_dataset.prefetch(1)

eval_dataset.take(1)

<DatasetV1Adapter shapes: ({input_dnn: (2000, 14), input_0: (2000, None, 8), input_1: (2000, None, 8)}, {y: (2000,), rnn_output_0: (2000, 14), rnn_output_1: (2000, 14)}), types: ({input_dnn: tf.float32, input_0: tf.float32, input_1: tf.float32}, {y: tf.float32, rnn_output_0: tf.float32, rnn_output_1: tf.float32})>

## Create the model

In [12]:
from tensorflow.keras.utils import plot_model
MASKING_VALUE = 666

input_dnn = keras.Input(shape =(14,), name='input_dnn' )
input_0 = keras.Input(shape = ( MAX_MOL_ATOMS_NB, 8), name = 'input_0' )
input_1 = keras.Input(shape = ( MAX_MOL_ATOMS_NB, 8), name = 'input_1' )
input_rnn = keras.Input(shape = ( MAX_MOL_ATOMS_NB, 8))

def create_model():

    dnn = layers.BatchNormalization()(input_dnn)
    dnn = layers.Dense(32, activation='relu')(dnn)
    dnn = layers.Dropout(0.5)(dnn)
    dnn_output = layers.Dense(32, activation='relu')(dnn)

    # Create a model for the rnn, shared for both atoms 0 and 1
    
    rnn = layers.Masking(mask_value=MASKING_VALUE, input_shape=(MAX_MOL_ATOMS_NB, 8))(input_rnn) 
    rnn = layers.LSTM(32, return_sequences=True, dropout = 0.5)(rnn)
    rnn_encoding = layers.LSTM(32, return_sequences=False, dropout = 0.5)(rnn)
    rnn_output = layers.Dropout(0.5)(rnn_encoding)

    model_rnn = keras.Model(input_rnn, outputs = [rnn_output], name = 'model_rnn')

    rnn_encoding_0 = model_rnn(input_0)
    rnn_output_0 = layers.Dense(32, activation='relu')(rnn_encoding_0)
    rnn_output_0 = layers.Dropout(0.5)(rnn_output_0)
    output_0 = layers.Dense(14, activation='linear', name = 'rnn_output_0')(rnn_output_0)
    
    rnn_encoding_1 = model_rnn(input_1)
    rnn_output_1 = layers.Dense(32, activation='relu')(rnn_encoding_1)
    rnn_output_1 = layers.Dropout(0.5)(rnn_output_1)
    output_1 = layers.Dense(14, activation='linear', name = 'rnn_output_1')(rnn_output_1)

    y_all = layers.concatenate([rnn_encoding_0, rnn_encoding_1, dnn_output])
    y_all = layers.Dropout(0.5)(y_all)
    y_all = layers.Dense(32, activation='relu')(y_all)
    y_output = layers.Dense(1, activation='linear', name = 'y')(y_all)

    return keras.Model(inputs = [input_dnn, input_0, input_1], outputs = [y_output, output_0, output_1], name = 'model_global')

model = create_model()

# Customer score to calculate the competition score

def custom_loss_wrapper(input_tensor):
    
    def custom_loss(y_true, y_pred):
        
        # Coupling one-hot
        c = tf.slice(input_tensor, [0,0], [-1,8])
        
        # without explicit broadcasting
        absolute_error_pred = tf.math.abs(tf.subtract(y_true,y_pred))
        absolute_error_per_type = tf.multiply(c, absolute_error_pred)
        
        zerocount_per_type = tf.math.count_nonzero(absolute_error_per_type, axis = 0)
        sum_per_type = tf.math.reduce_sum(absolute_error_per_type, axis = 0) 
        
        zerocount_per_type = tf.boolean_mask(zerocount_per_type, tf.greater(zerocount_per_type,0))
        sum_per_type = tf.boolean_mask(sum_per_type, tf.greater(sum_per_type,0.0))
        
        mean_per_type = sum_per_type / tf.cast(zerocount_per_type, tf.float32)
        mean_per_type = tf.math.log(mean_per_type)
        score = tf.math.reduce_mean(mean_per_type)     
        return score
    
    return custom_loss

model.summary()

W0716 08:00:38.559914  8180 deprecation.py:323] From D:\Program_Files\Anaconda3\envs\tensorflow2\lib\site-packages\tensorflow\python\keras\backend.py:3868: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Model: "model_global"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_dnn (InputLayer)          [(None, 14)]         0                                            
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 14)           56          input_dnn[0][0]                  
__________________________________________________________________________________________________
dense (Dense)                   (None, 32)           480         batch_normalization[0][0]        
__________________________________________________________________________________________________
input_0 (InputLayer)            [(None, 29, 8)]      0                                            
_______________________________________________________________________________________

In [13]:
model_save = ".\\models\\weights-improvement-2019715-34-1242.13.hdf5"
model.load_weights(model_save)

In [ ]:
from tensorflow.keras.callbacks import Callback

from datetime import datetime
now = datetime.now()

adam = keras.optimizers.Adam()
model.compile(loss=[custom_loss_wrapper(input_dnn), 'mean_squared_error', 'mean_squared_error'], 
              loss_weights = [100,1,1], optimizer=adam)

# Define the Keras TensorBoard callback.
#logdir=".\\logs\\fit\\" + datetime.now().strftime("%Y%m%d-%H%M%S")
#tensorboard_callback = keras.callbacks.TensorBoard(log_dir=logdir)

# checkpoint
filepath=".\\models\\weights-improvement-" + str(now.year) + str(now.month) + str(now.day) + "-{epoch:02d}-{val_loss:.2f}.hdf5"
checkpoint = keras.callbacks.ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

# Fit the model
history = model.fit(train_dataset, epochs=35, validation_data=eval_dataset, 
                    steps_per_epoch = 100, validation_steps=200, callbacks = callbacks_list)


Epoch 1/35
 99/100 [============================>.] - ETA: 2s - loss: 2967.6295 - y_loss: 1.3828 - rnn_output_0_loss: 665.8200 - rnn_output_1_loss: 2163.5315
Epoch 00001: val_loss improved from inf to 1675.13277, saving model to .\models\weights-improvement-2019716-01-1675.13.hdf5
100/100 [==============================] - 226s 2s/step - loss: 2966.5793 - y_loss: 1.3812 - rnn_output_0_loss: 666.1855 - rnn_output_1_loss: 2162.2695 - val_loss: 1675.1328 - val_y_loss: 1.0872 - val_rnn_output_0_loss: 302.6292 - val_rnn_output_1_loss: 1263.7794
Epoch 2/35
 99/100 [============================>.] - ETA: 1s - loss: 2587.3625 - y_loss: 1.2559 - rnn_output_0_loss: 613.2243 - rnn_output_1_loss: 1848.5436
Epoch 00002: val_loss did not improve from 1675.13277
100/100 [==============================] - 206s 2s/step - loss: 2587.9632 - y_loss: 1.2559 - rnn_output_0_loss: 613.4869 - rnn_output_1_loss: 1848.8910 - val_loss: 1774.5339 - val_y_loss: 1.1188 - val_rnn_output_0_loss: 387.8045 - val_rnn_out

In [ ]:
%tensorboard --logdir logs/fit --port 8088
#http://g751-antoine:8088/

In [ ]:
%load_ext tensorboard
!rm -rf ./logs/ 
%tensorboard --logdir logs/fit --port 8088
#http://g751-antoine:8088/

In [ ]:
model_save = ".\\models\\weights-final-2019712-04-31.99.hdf5"
model.save_weights(model_save)

In [ ]:
%tensorboard --logdir logs/fit --port 8088

In [ ]:
%kill 8168

In [ ]:
from tensorflow.keras.utils import plot_model
MASKING_VALUE = 666

input_dnn = keras.Input(shape =(14,), name='input_dnn' )

def create_simple_model():

    dnn = layers.BatchNormalization()(input_dnn)
    dnn = layers.Dense(32, activation='relu')(dnn)
    dnn = layers.Dropout(0.5)(dnn)
    dnn_output = layers.Dense(32, activation='relu')(dnn)
    y_all = layers.Dropout(0.5)(dnn_output)
    y_all = layers.Dense(32, activation='relu')(y_all)
    y_all = layers.Dropout(0.5)(y_all)
    y_output = layers.Dense(1, activation='linear', name = 'y')(y_all)
    
    return keras.Model(inputs = [input_dnn], outputs = [y_output], name = 'simple_model')

simple_model = create_simple_model()

def custom_loss_wrapper(input_tensor):
    
    def custom_loss(y_true, y_pred):
        
        # Coupling one-hot
        c = tf.slice(input_tensor, [0,0], [-1,8])
        
        # without explicit broadcasting
        absolute_error_pred = tf.math.abs(tf.subtract(y_true,y_pred))
        absolute_error_per_type = tf.multiply(c, absolute_error_pred)
        
        zerocount_per_type = tf.math.count_nonzero(absolute_error_per_type, axis = 0)
        sum_per_type = tf.math.reduce_sum(absolute_error_per_type, axis = 0) 
        
        zerocount_per_type = tf.boolean_mask(zerocount_per_type, tf.greater(zerocount_per_type,0))
        sum_per_type = tf.boolean_mask(sum_per_type, tf.greater(sum_per_type,0.0))
        
        mean_per_type = sum_per_type / tf.cast(zerocount_per_type, tf.float32)
        mean_per_type = tf.math.log(mean_per_type)
        score = tf.math.reduce_mean(mean_per_type)     
        return score
    
    return custom_loss

In [ ]:
from tensorflow.keras.callbacks import Callback

from datetime import datetime
now = datetime.now()

adam = keras.optimizers.Adam()
simple_model.compile(loss=[custom_loss_wrapper(input_dnn)], optimizer=adam)

# Define the Keras TensorBoard callback.
logdir=".\\logs\\fit\\" + datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = keras.callbacks.TensorBoard(log_dir=logdir)

# checkpoint
filepath="./models/weights-improvement-" + str(now.year) + str(now.month) + str(now.day) + "-{epoch:02d}-{val_loss:.2f}.hdf5"
checkpoint = keras.callbacks.ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [tensorboard_callback]

# Fit the model
history = simple_model.fit(train_dataset, epochs=5, validation_data=eval_dataset, 
                    steps_per_epoch = 500, validation_steps=100, callbacks = callbacks_list)

### DNN model

In [ ]:
import matplotlib.pyplot as plt

# list all data in history
print(history.history.keys())

# summarize history for loss
plt.plot(history.history['loss'])
#plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'eval'], loc='upper left')
plt.show()

#### Questions: 
* how do we bring the dataset to the model with several inputs, and outputs?

In [ ]:
model.evaluate(eval_dataset)

## Submissions

Submissions are evaluated on the Log of the Mean Absolute Error, calculated for each scalar coupling type, and then averaged across types, so that a 1% decrease in MAE for one type provides the same improvement in score as a 1% decrease for another type.

score=1T∑t=1Tlog(1nt∑i=1nt|yi−yi^|)
Where:

T is the number of scalar coupling types
nt is the number of observations of type t
yi is the actual scalar coupling constant for the observation
yi^ is the predicted scalar coupling constant for the observation
For this metric, the MAE for any group has a floor of 1e-9, so that the minimum (best) possible score for perfect predictions is approximately -20.7232.

## Annex

#### Features description

* dipole_moments.csv - contains the molecular electric dipole moments. These are three dimensional vectors that indicate the charge distribution in the molecule. The first column (molecule_name) are the names of the molecule, the second to fourth column are the X, Y and Z components respectively of the dipole moment.
* magnetic_shielding_tensors.csv - contains the magnetic shielding tensors for all atoms in the molecules. The first column (molecule_name) contains the molecule name, the second column (atom_index) contains the index of the atom in the molecule, the  third to eleventh columns contain the XX, YX, ZX, XY, YY, ZY, XZ, YZ and ZZ elements of the tensor/matrix respectively.
* mulliken_charges.csv - contains the mulliken charges for all atoms in the molecules. The first column (molecule_name) contains the name of the molecule, the second column (atom_index) contains the index of the atom in the molecule, the third column (mulliken_charge) contains the mulliken charge of the atom.
* potential_energy.csv - contains the potential energy of the molecules. The first column (molecule_name) contains the name of the molecule, the second column (potential_energy) contains the potential energy of the molecule.
* scalar_coupling_contributions.csv - The scalar coupling constants in train.csv (or corresponding files) are a sum of four terms. scalar_coupling_contributions.csv contain all these terms. The first column (molecule_name) are the name of the molecule, the second (atom_index_0) and third column (atom_index_1) are the atom indices of the atom-pair, the fourth column indicates the type of coupling, the fifth column (fc) is the Fermi Contact contribution, the sixth column (sd) is the Spin-dipolar contribution, the seventh column (pso) is the Paramagnetic spin-orbit contribution and the eighth column (dso) is the Diamagnetic spin-orbit contribution.

### Test set to validate the code

In [ ]:
ut_coupling = pd.DataFrame({
    'molecule_name': ['mol_a', 'mol_b', 'mol_a', 'mol_b', 'mol_b', 'mol_c'],
    'atom_index_0':[ 0, 0, 1, 1, 1, 0],
    'atom_index_1': [ 1, 1, 2, 2, 3, 1],
    'scalar_coupling_constant': [5, 7, 6, 8, 9, 10],
    'coupling_type_1JHC':[ 1, 0, 0, 0, 0, 0], 
    'coupling_type_1JHN':[ 0, 1, 0, 0, 0, 0], 
    'coupling_type_2JHC':[ 0, 0, 1, 0, 0, 0], 
    'coupling_type_2JHH':[ 0, 0, 0, 1, 0, 0], 
    'coupling_type_2JHN':[ 0, 0, 0, 0, 1, 0], 
    'coupling_type_3JHC':[ 0, 0, 0, 0, 0, 1],
    'coupling_type_3JHH':[ 0, 0, 0, 0, 0, 0], 
    'coupling_type_3JHN':[ 0, 0, 0, 0, 0, 0]
})

ut_coupling.head(6)

In [ ]:
ut_molecules = pd.DataFrame({
    'molecule_name': ['mol_a', 'mol_a', 'mol_a', 'mol_b', 'mol_b', 'mol_b', 'mol_b', 'mol_b', 'mol_c', 'mol_c'],
    'atom_index':[ 0, 1, 2, 0, 1, 2, 3, 4, 0, 1],
    'x': [ -0.5, 0.5, 1, -2, -1, 0, 1, 2, 0.5, 1.5  ],
    'y': [ 0, 0, 0, 0, 0, 0, 0, 0, 0, 0  ],
    'z': [ -0.5, 0.5, 1, -2, -1, 0, 1, 2, 0.5, 1.5  ],
    'atom_C': [1,0,0,0,0,1,0,0,0,0],
    'atom_F': [0,1,0,0,0,0,1,0,0,0],
    'atom_H': [0,0,1,0,0,0,0,1,0,0], 
    'atom_N': [0,0,0,1,0,0,0,0,1,0],
    'atom_O': [0,0,0,0,1,0,0,0,0,1]
})

ut_molecules.head(10)